# Чтение данных

In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [2]:
df_list = []

for i in range(12):
    df_list.append(
        pd.read_csv(f"../collecting/columns_headers/headers_{i}.csv", sep="⇧", engine="python", quotechar='"', on_bad_lines="warn")
    )
df = pd.concat(df_list, axis=0)
df

Skipping line 49505: '⇧' expected after '"'
Skipping line 49506: '⇧' expected after '"'
Skipping line 49507: '⇧' expected after '"'
Skipping line 345854: field larger than field limit (131072)


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
3,dataset/Dataset/data/4022432/table_0,1,Unnamed: 1
4,dataset/Dataset/data/4022432/table_0,2,Русское название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


# Research
---
Проблемные столбцы (скорее всего не будем использовать):
- Столбцы по типу: "ΔHкип, кДж·моль−1"
- Столбцы использующиеся для группировки каких-либо значений по времени (месяцу, году и т.п.)
- Столбцы без имени
- Столбцы, название которых не имеет смысла (числа и т.п.)
- Аббревиатуры и сокращения

### Столбцы без названия = 164500
их можно отправить в тестовую выборку, чтобы не терять информацию. (нужно еще посмотреть, что внутри них обычно лежит)

### Без имени
можно в тест

## .mv-parser... = 15661
обрезать только осмысленный текст

---

In [3]:
df["column_name"].value_counts()

column_name
Год                    220601
Место                   87167
Роль                    78957
Команда                 67698
Название                64939
                        ...  
Со-производитель(и)         1
Области применения          1
51 220                      1
1999—2001                   1
1724                        1
Name: count, Length: 125673, dtype: int64

## Столбцы без названия

In [4]:
unnamed_columns = df["column_name"].apply(lambda x: str(x).startswith("Unnamed"))
df[unnamed_columns]

,table_id,column_id,column_name
3,dataset/Dataset/data/4022432/table_0,1,Unnamed: 1
215,dataset/Dataset/data/8128699/table_0,1,Unnamed: 1
225,dataset/Dataset/data/7786517/table_0,1,Unnamed: 1
267,dataset/Dataset/data/3595819/table_2,0,Unnamed: 0
324,dataset/Dataset/data/1802083/table_7,0,Unnamed: 0
...,...,...,...
411160,dataset/Dataset/data/8352540/table_7,1,Unnamed: 1
411268,dataset/Dataset/data/2584082/table_1,2,Unnamed: 2
411275,dataset/Dataset/data/2584082/table_1,9,Unnamed: 9
411289,dataset/Dataset/data/980012/table_0,0,Unnamed: 0


## Столбцы содержащие в названии: ".mw-parser..."

In [5]:
mv_parser = df["column_name"].apply(lambda x: str(x).strip().startswith(".mw-parser"))
df[mv_parser]

,table_id,column_id,column_name
52,dataset/Dataset/data/4659559/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...
605,dataset/Dataset/data/5220971/table_15,1,.mw-parser-output .ts-comment-commentedText{bo...
1437,dataset/Dataset/data/5346766/table_1,0,.mw-parser-output .ts-comment-commentedText{bo...
1777,dataset/Dataset/data/4747431/table_1,0,.mw-parser-output .ts-comment-commentedText{bo...
2040,dataset/Dataset/data/7955239/table_5,0,.mw-parser-output .ts-comment-commentedText{bo...
...,...,...,...
410876,dataset/Dataset/data/5375371/table_0,1,.mw-parser-output .ts-comment-commentedText{bo...
410941,dataset/Dataset/data/5372811/table_0,1,.mw-parser-output .ts-comment-commentedText{bo...
410988,dataset/Dataset/data/2554658/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...
411366,dataset/Dataset/data/5451199/table_0,0,.mw-parser-output .ts-comment-commentedText{bo...


### В назнваниях столбца содержится человекопонятное название, нужно отчистить мусорную информацию

In [6]:
mv_parser_humnareadable_names = df[mv_parser]["column_name"].apply(lambda x: str(x).split("}")[-1])
mv_parser_humnareadable_names.unique()

array(['М', 'МУ', 'Цвета участниц', 'Дата пожалования', '#', 'Ref.', 'ХП',
       '№', 'Жертвы', '7,5+7,5 км', 'Коэфф.', 'В', 'И', 'N', 'Коэф.',
       'Д / В', 'Год', 'П', '№ п/п', ' В', '09/10', 'Bib', 'Прим.',
       '02/03', 'Пер.', 'Сезоны', 'Категория', 'Кубок', 'С', 'И.', 'БК',
       'ТТХ', 'Уров.', 'HP', 'Поз', 'Тренер', 'КЧ', 'КП', 'Поз.', 'СУ',
       'Н', 'Доступ-ность', 'Гр.', 'ПЧ', '08/09', '11/12',
       'Country Songs[A]', '±', 'З', 'Должность / Титул', 'Ном.', 'ПП',
       'д/г', 'Этап', '15+15 км', ' От', 'СУ, км', ' И', 'К-во', 'GP',
       'Очки', 'И, С', 'П.', 'Численностьверующихмлн. человек', 'Период',
       'ГП[67]', 'Дата рождения', 'Нац', '+/-', '06/07', 'ед. изм.',
       'Оценка за трудность',
       'Доля учеников по обучению на русском и белорусском языках,\xa0%[42][39][43]:',
       '04/05', 'Ист.', 'Кр', 'ЧС', 'КНСО', 'Голосовало', 'Рез.', 'SP',
       'SP.1', 'Норвежское название', 'Оч', 'Л/Круги', 'Оценка D', 'RPG',
       'Б/Круги', 'Плотность(чел./

## Аббревиатуры / сокращения / меры (кг, кдж.)

In [7]:
# TODO: what should i do? -_-

## Заголовки, состоящие из одной буквы

In [8]:
no_word_in_header = df["column_name"].str.len() < 2
df[no_word_in_header]["column_name"].unique()

array(['№', 'И', 'В', 'Н', 'П', '±', 'О', 'М', 'Г', '#', 'N', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', '%', 'Ш', 'Л', 'A', 'B', '+',
       '−', '=', 'm', '0', '-', 'Р', 'I', 'R', 'З', 'С', 'Δ', 'G', 'W',
       'L', 'n', '«', '»', 'ښ', 'ږ', 'څ', 'ځ', 'ژ', 'ā', 'ū', 'ə', 'u',
       'i', 'a', 'e', 'o', 'Σ', 'F', 'D', 'P', 'Я', 'Т', 'А', '<', '>',
       '∧', 'T', 'Д', 'H', 'X', 'Y', 'Б', 'Z', 'M', 'Ф', 'ä', '學', 'ɔ',
       'ʊ', 'ɵ', 'ɮ', 'f', 's', 'ʃ', 't', 'j', 'r', 'h', '*', 'Э', 'V',
       '…', '→', '×', 'K', 'Ч', '_', 'ذ', 'ڈ', 'د', 'خ', 'ح', 'چ', 'ج',
       'ث', 'ٹ', 'ت', 'پ', 'ب', 'ا', 'Q', 'S', 'c', 'b', 'k', 'l', 'p',
       'w', 'x', 'y', 'Ж', '↕', 'C', 'E', '•', 'У', 'अ', 'आ', 'इ', 'ई',
       'उ', 'ऊ', 'ए', 'ऐ', 'ओ', 'औ', 'ऋ', 'ॠ', 'ऌ', 'ॡ', 'J', 'O', 'U',
       'я', 'К', 'Á', 'Ⱥ', 'Ć', 'Ȼ', '·', '∑', 'χ', 'd', 'é', 'g', 'ɨ',
       'q', 'ṟ', 'ʉ', 'v', 'z', "'", '^', 'с', 'क', 'ख', 'ग', 'घ', 'ङ',
       'Α', 'Β', 'Γ', 'Ε', 'Ϝ', 'Ζ', 'Η', 'Θ', 'Ι', 'Κ', 'Λ', 'Μ

# Очистка данных

---
1. Очистка мусора из столбцов, содержащих ".mv-parser.."
2. Выделение столбцов без заголовка
3. Замена специальных символов на слова (№, $, %, и т.п.)
4. Очистка символов не из русского алфавита
5. Удаление лишних пробелов с начала и конца заголовка
---

In [9]:
df_cleansed = df.copy()

## 1. Очистка мусора из столбцов

In [10]:
def clean_mv_parser(col_name: str) -> str:
    if col_name.strip().startswith(".mw-parser"):
        return col_name.split("}")[-1]
    return col_name

df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: clean_mv_parser(str(x)))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4965706/4965706 [00:02<00:00, 2407940.42it/s]


In [11]:
df_cleansed[mv_parser]

,table_id,column_id,column_name
52,dataset/Dataset/data/4659559/table_0,0,М
605,dataset/Dataset/data/5220971/table_15,1,МУ
1437,dataset/Dataset/data/5346766/table_1,0,М
1777,dataset/Dataset/data/4747431/table_1,0,М
2040,dataset/Dataset/data/7955239/table_5,0,М
...,...,...,...
410876,dataset/Dataset/data/5375371/table_0,1,И
410941,dataset/Dataset/data/5372811/table_0,1,И
410988,dataset/Dataset/data/2554658/table_0,0,Поз.
411366,dataset/Dataset/data/5451199/table_0,0,М


## 2. Выделение столбцов без заголовка

In [12]:
unnamed_columns = df_cleansed["column_name"].str.startswith("Unnamed")
df_cleansed = df_cleansed[~unnamed_columns]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
4,dataset/Dataset/data/4022432/table_0,2,Русское название
5,dataset/Dataset/data/4022432/table_0,3,Оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3. Замена специальных символов на слова

In [13]:
def remove_alphabetic_symbols(col_name: str) -> str:
    """TODO: just replace this symbols with space"""
    restricted_symbols = [
        "(", 
        ")", 
        "+", 
        "-", 
        "[", 
        "]", 
        "/", 
        ".", 
        ",", 
        "@", 
        "{", 
        "}", 
        "«", 
        "»", 
        "<", 
        ">", 
        ":", 
        ";", 
        "'",
        "*",
        "?",
        "!",
        "-",
        "_",
    ]
    return "".join(c for c in col_name if not c.isalpha() and not c.isdigit() and not c.isspace() and c not in restricted_symbols)

non_alpha = df_cleansed["column_name"].progress_apply(lambda x: remove_alphabetic_symbols(str(x)))
non_alpha.unique()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4801266/4801266 [00:04<00:00, 1005520.64it/s]


array(['', '№', '±', '#', '—', '%', '$', '\\№', '☉', '•', '−', '=', '""',
       '–', '⊙', '\\', '°', '·−', '́', '€', '\xad', '’%', '–%', '∅', '≈',
       '&', '|', '%%', '%%%%———————————', '\xad\xad', '∧', '••', '→↓',
       '₽', '̱', '½', '≥', '\x7f"``"\x7f', '·', '÷%', '…', '͡', '≡', '‰',
       '↓→', '——', '&&', '’', '=""', '̨', '’’', '→', '×', '~', '↓', '⋅',
       '№№', '°·', '£', '±%', '$$', '†', '—°', '↕', '̃', '•—', '%~', '––',
       'ं', 'ः', '%—', '‡', '×=', '====', '№№——№—№——№——№№—№—№№—', '्',
       '½½', '∑', '¶', '¥', '′', '%%%%%%', '||', '××', '^', '"', '−·',
       '⊕', '￼', '♂', '♀', '−−', '´', '˘', '———', '∆%', '（）', '%%%%%%%%',
       '¼', '̱̱', '%±', 'ꤦ', 'ꤧ', 'ꤨ', 'ꤩ', 'ꤪ', '★', '№\\',
       '\xad\xad\xad\xad\xad', '\xad\xad\xad\xad', '∙−', '∙∙−', '≤°',
       '≥°', '▼', 'ಂ', 'ಃ', '∞∞', '""""', '¬', '∞', '♁', '——————',
       '———————', '\xad\xad\xad', '……………………', '=%', '↑', '▼▲%',
       '№№—№—————————', '—%', '=−', '\u200e', '%№', '′′', '’—', '‘…’',
       '⠀'

In [14]:
# # - тоже номер или не всегда
# Много знаков равенства, скорее всего в столбце число лежит, можно тоже указать
rules = {
    "№": " номер ",
    "%": " процент ",
    "∆": " число ",
    "∑": " сумма ",
    "Σ": " сумма ",
    "$": " деньги ",
    "₽": " деньги ",
    "€": " деньги ",
    "¥": " деньги ",
    "₩": " деньги ",
    "£": " деньги ",
}

for rule, word in rules.items():
    df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).replace(rule, word))

df_cleansed

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4801266/4801266 [00:01<00:00, 2593427.10it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,Чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,Высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,Год
4,dataset/Dataset/data/4022432/table_0,2,Русское название
5,dataset/Dataset/data/4022432/table_0,3,Оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,Дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,Сезон
411441,dataset/Dataset/data/6206410/table_0,1,Короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,Произвольная программа


## 3.1. To lower

In [15]:
df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).lower())
df_cleansed

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4801266/4801266 [00:02<00:00, 2190293.86it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт (2019)
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


## 3.2 замена ё на е

In [16]:
df_cleansed["column_name"] = df_cleansed["column_name"].str.replace("ё", "е")

## 4. Удаление символов не из русского алфавита

In [17]:
# for i in df_cleansed.sort_values(by=["column_name"], ascending=False)["column_name"]:
#     print(i)

In [18]:
def remove_non_alphabetic_symbols(col_name: str) -> str:
    # return "".join(c for c in col_name if c.isalpha() or c == " ")
    # return "".join(c for c in col_name if "А" <= c <= "я" or c == " ")
    for c in col_name:
        if not ("а" <= c <= "я" or c == " "):
            col_name = col_name.replace(c, " ")
    return col_name
    # return "".join(c for c in col_name if "а" <= c <= "я" or c == " ")

df_cleansed["column_name"] = df_cleansed["column_name"].progress_apply(lambda x: remove_non_alphabetic_symbols(str(x)))
df_cleansed

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4801266/4801266 [00:03<00:00, 1229488.05it/s]


,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [19]:
len_le_2 = df_cleansed["column_name"].str.len() == 0
df_cleansed[len_le_2]

,table_id,column_id,column_name
80632,dataset/Dataset/data/8289176/table_1,7,
378460,dataset/Dataset/data/8113812/table_1,7,


## 5. Удаление пробелов с начала и конца заголовка столбца

In [20]:
df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lambda x: str(x).strip())
df_cleansed = df_cleansed[df_cleansed["column_name"] != ""]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 4801266/4801266 [00:01<00:00, 2746387.56it/s]


## 6. Удаление заголовков столбцов, используемых для группировки значений (по месяцу, году и т.п.)

In [21]:
group_by_columns = [
    "январь",
    "февраль",
    "март",
    "апрель",
    "май",
    "июнь",
    "июль",
    "август",
    "сентябрь",
    "октябрь",
    "ноябрь",
    "декабрь",
    "янв",
    "февр",
    "фев",
    "мар",
    "апр",
    "июн",
    "июл",
    "авг",
    "сент",
    "сен",
    "окт",
    "ноя",
    "нояб",
    "дек",
]

In [22]:
s = 0
for col in group_by_columns:
    s += df_cleansed[df_cleansed["column_name"] == col].shape[0]
s

137387

In [23]:
for col in group_by_columns:
    group_by_mask = df_cleansed["column_name"] == col
    df_cleansed = df_cleansed[~group_by_mask]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [24]:
(df_cleansed["column_name"] == "фев").sum()

0

## 7. Удаление заголовков длиной <= 2

TODO: split count max len of word; if <= 2 -> drop

In [25]:
len_le_2 = df_cleansed["column_name"].str.len() <= 2
df_cleansed[len_le_2]

,table_id,column_id,column_name
47,dataset/Dataset/data/7873710/table_0,2,пу
52,dataset/Dataset/data/4659559/table_0,0,м
54,dataset/Dataset/data/4659559/table_0,2,и
55,dataset/Dataset/data/4659559/table_0,3,в
56,dataset/Dataset/data/4659559/table_0,4,н
...,...,...,...
411413,dataset/Dataset/data/3818327/table_0,2,и
411414,dataset/Dataset/data/3818327/table_0,3,в
411415,dataset/Dataset/data/3818327/table_0,4,н
411416,dataset/Dataset/data/3818327/table_0,5,п


In [26]:
df_cleansed = df_cleansed[~len_le_2]
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
4,dataset/Dataset/data/4022432/table_0,2,русское название
5,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
411439,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
411440,dataset/Dataset/data/6206410/table_0,0,сезон
411441,dataset/Dataset/data/6206410/table_0,1,короткая программа
411442,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [27]:
df_cleansed = df_cleansed.reset_index().drop(["index"], axis=1)
df_cleansed

,table_id,column_id,column_name
0,dataset/Dataset/data/7779522/table_0,0,чарт
1,dataset/Dataset/data/7779522/table_0,1,высшаяпозиция
2,dataset/Dataset/data/4022432/table_0,0,год
3,dataset/Dataset/data/4022432/table_0,2,русское название
4,dataset/Dataset/data/4022432/table_0,3,оригинальное название
...,...,...,...
3381676,dataset/Dataset/data/35185/table_7,6,дата выхода из союза
3381677,dataset/Dataset/data/6206410/table_0,0,сезон
3381678,dataset/Dataset/data/6206410/table_0,1,короткая программа
3381679,dataset/Dataset/data/6206410/table_0,2,произвольная программа


In [28]:
from random import randint

rnd = randint(0, df_cleansed.shape[0])
df_cleansed.loc[rnd:rnd+10]

,table_id,column_id,column_name
2459823,dataset/Dataset/data/939407/table_65,0,имя
2459824,dataset/Dataset/data/939407/table_65,1,пол
2459825,dataset/Dataset/data/939407/table_65,2,вид
2459826,dataset/Dataset/data/939407/table_65,3,роль в фильмах
2459827,dataset/Dataset/data/7012586/table_0,1,команда
2459828,dataset/Dataset/data/8833481/table_1,0,номер
2459829,dataset/Dataset/data/8833481/table_1,1,город
2459830,dataset/Dataset/data/8833481/table_1,2,область
2459831,dataset/Dataset/data/8833481/table_1,3,категория
2459832,dataset/Dataset/data/8833481/table_1,5,номер


# Сохранение очищенного датафрейма

In [29]:
import numpy as np

number_of_chunks = 10
for id, chunk in enumerate(np.array_split(df_cleansed, number_of_chunks)):
    chunk.to_csv(f"./cleansed/data_{id}.csv", sep=";", index=False)

/home/kirilltobola/Dev/github.com/isp/cta/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


# Лемматизация

Дальнейшая разметка строится на предположении, что первое существительное в заголовке столбца отражает ту сущность, которая в нем содержится

In [34]:
import spacy

nlp = spacy.load("ru_core_news_sm", disable=['parser', 'ner'])

def lemmatize_word(word: str) -> str|None:
    for token in nlp(word):
        if token.pos_ == "NOUN":
            return token.lemma_
    return None

In [36]:
from pathlib import Path

for chunk_id in range(number_of_chunks):
    if not Path(f"./lemmatized/lemmatized_{chunk_id}.csv").exists():
        df_cleansed = pd.read_csv(f"./cleansed/data_{chunk_id}.csv", sep=";")
    
        df_cleansed.loc[:, "column_name"] = df_cleansed["column_name"].progress_apply(lemmatize_word)
        df_cleansed.to_csv(f"./lemmatized/lemma_{chunk_id}.csv", sep=";", index=False)